In [40]:
import numpy as np
import pandas as pd
from datetime import datetime
import os

In [41]:
base_path_DPMU = "..\\..\\data\\summer\\targets\\trimmed_targets"
all_months_DPMU = {} ##{mm:{a:, b:, c:}, mm:{a:,b:,c:}}

include_kmeans = 0
include_PCA = 0
include_holidays = 1
include_wknds = 1
include_phase = 1

# Input (DPMU) Preprocessing (sends to X.csv)

In [42]:
#modifies raw (trimmed) csv to add time features and process the various phases/houses
def modify_csv(month_phase_csv, month_func, year_user):
    ##time modifications
    time = month_phase_csv['# timestamp']
    time = pd.Series(time)
    time = time.str.split(" ")
    for index in range(len(time)):
        del time[index][2]
    time
    month = []
    hour = []
    for index in range(len(time)):
        month.append(datetime.strptime(time[index][0],'%Y-%m-%d'))
        hour.append(datetime.strptime(time[index][1],'%H:%M:%S'))
    date = []
    timestamp = []

    for index in range(len(time)):
        for jindex in range(len(time[index])):
            if "-" in time[index][jindex]:
                date.append(time[index][jindex].split("-"))
            if ":" in time[index][jindex]:
                timestamp.append(time[index][jindex].split(":"))
    date = np.array(date)
    timestamp = np.array(timestamp)
    #separating the individual values into different list
    year = []
    month = []
    day = []
    for y in range(len(date)):
        year.append(date[y][0])
        month.append(date[y][1])
        day.append(date[y][2])

    #do special processing for the hours and minutes
    day_min = []
    for y in range(len(timestamp)):
        hour_min_added = int(timestamp[y][0]) * 60
        hour_min_added = hour_min_added + int(timestamp[y][1])
        day_min.append(hour_min_added) 

    ##code to overwrite 'January' outputs for csv's    
    year = [year_user] * len(date)
    month = [month_func] * len(date)
        
    time_df = pd.DataFrame()
    time_df["year"] = year
    time_df["month"] = month
    time_df["day"] = day
    

    time_df["hour_min"] = day_min
        
    month_phase_mod = month_phase_csv.drop(['Meter5', '# timestamp'], axis=1)
    month_phase_mod = pd.concat([month_phase_mod,time_df], axis=1)
    #print (month_phase_mod)
    return month_phase_mod

In [43]:
#modifies the global all_months_DPMU. When this is over, that array will have all the necessary information to be concatenated into a giant array.
def sep_DPMU_features():
    #print (base_path_DPMU)
    for month in os.listdir(base_path_DPMU):
        month_path = os.path.join(base_path_DPMU,month)
        month_dict = {}
        for multi_voltage_output in os.listdir(month_path):
            full_csv_path = os.path.join(month_path,multi_voltage_output)
            #print(full_csv_path)
            cut_voltage = multi_voltage_output[:-4]
            #print(cut_voltage)
            letter = cut_voltage[-1:]
            #print(letter)
            dpmu_phase_month = "dpmu_phase" + letter + "_" + month
            raw_csv = pd.read_csv(full_csv_path)
            month_dict[letter] = modify_csv(raw_csv, month, 2018)
        #add month dictionary
        all_months_DPMU[month]=month_dict
    #return all the months it found in the base dir
    return list(all_months_DPMU.keys())
    

In [44]:
def DPMU(phase, dpmu_data, letter):
    phase_letter = ord(letter) - 65
    #print(phase_letter)
    for (columnName, columnData) in phase.iteritems():
        i=0
        if 'Meter' in columnName: ##meters
            split = columnName.split('Meter')
            house_name = f'Meter{letter}_'+ split[1]
            i+=1
            dpmu_values = pd.Series(columnData.values).str.replace("i", "j").apply(lambda x: abs(complex(x)))
            dpmu_data[house_name] = dpmu_values
        elif 'node' in columnName : ##nodes
            split_column_names = columnName.split('_')
            house_name = 'house'+str(split_column_names[phase_letter])
            dpmu_values = pd.Series(columnData.values).str.replace("i", "j").apply(lambda x: abs(complex(x)))
            dpmu_data[house_name] = dpmu_values
        else: ##timestamps, day_min
            if letter == 'A':
                dpmu_data[columnName] = pd.Series(columnData.values)
    return dpmu_data
    

In [45]:
def concat_all_DPMU():
    all_months_to_concat = []
    for month in all_months_DPMU:
        #print(month)
        dpmu_data_month = pd.DataFrame()
        for phase in all_months_DPMU[month]:
            #print (phase)
            dpmu_data_month = DPMU(all_months_DPMU[month][phase.upper()], dpmu_data_month, phase.upper())
        all_months_to_concat.append(dpmu_data_month)
    input_dataframe_DPMU = pd.DataFrame() ##concatenates all months, directly to input
    input_dataframe_DPMU = pd.concat(all_months_to_concat, ignore_index = True)
    return (input_dataframe_DPMU)
    
        

In [29]:
def sep_DPMU_clusters_kmeans(in_csv): #simoni
    return kmeans_df

SyntaxError: invalid syntax (Temp/ipykernel_1268/911446617.py, line 1)

In [30]:
def sep_DPMU_clusters_PCA(in_csv): #simoni
    return pca_df

SyntaxError: invalid syntax (Temp/ipykernel_1268/500842341.py, line 1)

In [ ]:
def phase_clustering(in_csv): #swetha
    
    return phase_clustering_df
    

In [ ]:
#return df modified to include flags for holidays
def add_holidays(in_csv): #swetha
    return holiday_df

In [ ]:
#return df modified to include flags for weekend
def add_weekends(in_csv): #swetha
    return wknd_df

In [46]:
##all_months_DPMU will have a dictionary of csv's for each month, each phase
sep_DPMU_features()

['aug', 'july', 'june', 'sept']

In [47]:
##input_dataframe_DPMU will have large dataframe of input data
send_input_to_csv = pd.DataFrame()
send_input_to_csv = concat_all_DPMU()

In [48]:
send_input_to_csv

,MeterA_2,MeterA_1,MeterA_3,MeterA_4,housenode1,housenode4,housenode7,housenode13,housenode10,housenode22,...,MeterC_3,MeterC_4,house3,house6,house9,house15,house12,house24,house21,house18
0,2376.409889,2378.388636,2361.400016,2357.538734,2363.994761,2376.991693,2375.718498,2352.767045,2361.400016,2361.418434,...,2347.430794,2343.286737,2366.926658,2366.504003,2366.229047,2337.377650,2347.430794,2344.827519,2347.430794,2342.868207
1,2380.907281,2382.797347,2371.481343,2369.147260,2369.150113,2380.564999,2378.824397,2364.411194,2371.481343,2371.381328,...,2350.315321,2346.584561,2368.194516,2368.069600,2367.731763,2340.824483,2350.315321,2347.744618,2350.315321,2346.217019
2,2377.256861,2379.415364,2366.150157,2362.635628,2365.298299,2377.165092,2375.486746,2355.295777,2366.150157,2365.965524,...,2347.366674,2343.390828,2368.214272,2367.751656,2367.378270,2337.614698,2347.366674,2344.726159,2347.366674,2343.008353
3,2374.778306,2376.835701,2360.276611,2355.968593,2363.462683,2374.634968,2372.936101,2347.306906,2360.276611,2360.103638,...,2346.915101,2342.973333,2368.147718,2367.509425,2367.188528,2337.128129,2346.915101,2344.249188,2346.915101,2342.826055
4,2374.294670,2376.486046,2360.364542,2356.340245,2361.685581,2374.155190,2372.415367,2348.156111,2360.364542,2360.259043,...,2350.219551,2346.321728,2370.321920,2369.072138,2368.743645,2340.504174,2350.219551,2347.680284,2350.219551,2346.162014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175675,2383.647127,2383.949621,2364.816449,2361.909454,2381.794340,2384.738434,2384.456407,2357.466568,2364.816449,2365.072928,...,2387.879512,2387.002581,2389.002002,2388.000478,2388.188000,2385.968149,2387.879512,2387.750663,2387.879512,2386.800404
175676,2384.341416,2384.643960,2366.016259,2363.111796,2382.212336,2385.452623,2385.210038,2358.636611,2366.016259,2366.251903,...,2381.532573,2380.660222,2385.728215,2384.553452,2384.738626,2379.636839,2381.532573,2381.440251,2381.523886,2380.458055
175677,2388.071887,2388.354823,2373.152195,2370.222813,2385.795089,2389.302478,2389.148854,2365.685649,2373.152195,2373.062116,...,2370.071818,2369.202664,2379.323677,2378.633987,2378.813187,2368.182870,2370.071818,2369.865101,2370.063124,2369.014147
175678,2388.068581,2388.391098,2373.609684,2370.815485,2385.984918,2389.277919,2389.113869,2366.339327,2373.609684,2373.440121,...,2366.610109,2365.695425,2378.270558,2377.354002,2377.516891,2364.671980,2366.610109,2366.403122,2366.610109,2365.488313


In [49]:
list(send_input_to_csv.columns)

['MeterA_2',
 'MeterA_1',
 'MeterA_3',
 'MeterA_4',
 'housenode1',
 'housenode4',
 'housenode7',
 'housenode13',
 'housenode10',
 'housenode22',
 'housenode19',
 'housenode16',
 'year',
 'month',
 'day',
 'hour_min',
 'MeterB_2',
 'MeterB_1',
 'MeterB_3',
 'MeterB_4',
 'house2',
 'house5',
 'house8',
 'house14',
 'house11',
 'house23',
 'house20',
 'house17',
 'MeterC_2',
 'MeterC_1',
 'MeterC_3',
 'MeterC_4',
 'house3',
 'house6',
 'house9',
 'house15',
 'house12',
 'house24',
 'house21',
 'house18']

In [51]:
send_input_to_csv = send_input_to_csv[['MeterA_1', 'MeterA_2', 'MeterA_3', 
                                       'MeterB_1', 'MeterB_2', 'MeterB_3',
                                      'MeterC_1', 'MeterC_2', 'MeterC_3',
                                      'housenode1','housenode4','housenode7',
                                       'housenode13','housenode10','housenode22',
                                       'housenode19','housenode16',
                                      'house2','house5', 'house8','house14',
                                       'house11','house23','house20','house17',
                                       'house3','house6','house9','house15','house12',
                                       'house24','house21','house18',
                                       'year','month','day','hour_min']]

# No clustering

In [52]:
#send_input_to_csv['hour_min']

0            0
1            1
2            2
3            3
4            4
          ... 
175675    1435
175676    1436
175677    1437
175678    1438
175679    1439
Name: hour_min, Length: 175680, dtype: int64

send_input_to_csv.to_csv('X.csv')

# K-means, PCA, phase, holidays, weekends

In [ ]:
modified_input = pd.DataFrame()
csv_name = "X"

if (include_kmeans):
    modified_input = sep_DPMU_clusters_kmeans(send_input_to_csv)
    csv_name = csv_name + "_kmeans"

if(include_PCA):
    modified_input = sep_DPMU_clusters_PCA(send_input_to_csv)
    csv_name = csv_name + "_pca"

if(include_phase):
    modified_input = phase_clustering(send_input_to_csv)
    csv_name = csv_name + "_phase"
    
if(include_holidays):
    modified_input = add_holidays(send_input_to_csv)
    csv_name = csv_name + "_holidays"
    
if(include_wknds):
    modified_input = add_weekends(send_input_to_csv)
    csv_name = csv_name + "_weekends"


In [ ]:
csv_name = csv_name = ".csv"
modified_input.to_csv(csv_name)

# Output (Grid) Preprocessing (sends to Y.csv)

In [14]:
base_path_Player = "..\\..\\data\\summer\\input_csv"

In [20]:
def set_output():
    month_players = []
    for month in os.listdir(base_path_Player):
        month_path = os.path.join(base_path_Player,month)
        input_player_df = pd.DataFrame()
        for filename in os.listdir(month_path):
            if str(filename).startswith('25.csv'):
                continue
            player = pd.read_csv(os.path.join(month_path, filename))
            player['grid'] = player['grid'].str.replace("i", "j").apply(lambda x: complex(x))
            node_id = player['id'].loc[0]
            column_name_p = 'p_'+str(node_id)
            column_name_q = 'q_'+str(node_id)
        #change .head() value to target dataframe row number
            input_player_df[column_name_p] = player['grid'].apply(lambda x: x.real).head(44640)
            input_player_df[column_name_q] = player['grid'].apply(lambda x: x.imag).head(44640)
        month_players.append(input_player_df)
    output_dataframe_Player = pd.DataFrame() ##concatenates all months, directly to input
    output_dataframe_Player = pd.concat(month_players, ignore_index = True)
    return (output_dataframe_Player)
    
    

In [21]:
##input_dataframe_DPMU will have large dataframe of input data
send_targets_to_csv = pd.DataFrame()
send_targets_to_csv = set_output()

In [22]:
send_targets_to_csv

,p_1,q_1,p_10,q_10,p_11,q_11,p_12,q_12,p_13,q_13,...,p_5,q_5,p_6,q_6,p_7,q_7,p_8,q_8,p_9,q_9
0,43770.0,18273.479083,49815.0,19677.968025,6210.0,2993.529277,7680.0,2977.205699,61020.0,25629.453761,...,53790.0,22680.824631,67515.0,26725.875676,29070.0,12766.083147,6045.0,2538.376656,5310.0,1768.574303
1,43800.0,16056.165241,49830.0,21169.506826,6210.0,2503.047231,7410.0,3187.775742,60975.0,24463.259976,...,53640.0,18591.367838,67395.0,24229.587465,39240.0,16830.434105,6090.0,2080.689498,5295.0,1960.017239
2,43740.0,16725.190261,49980.0,18571.507872,6210.0,2406.866873,7410.0,3368.241776,95295.0,34245.168859,...,53670.0,17841.890713,67035.0,23688.227789,39525.0,14630.069312,6165.0,2053.104821,5265.0,2332.545638
3,43695.0,14533.893706,49920.0,21483.136368,6210.0,2552.831448,7410.0,2727.305834,111255.0,45610.401766,...,53565.0,25357.082993,67005.0,25112.807910,39675.0,15125.471091,8325.0,3691.961411,5280.0,1817.387203
4,43635.0,18926.971373,49875.0,19129.801554,6210.0,2285.612698,8385.0,2785.115072,105225.0,42562.705246,...,53520.0,24179.304713,67050.0,27466.505811,39645.0,16417.639968,7950.0,2996.814233,5265.0,2177.816589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175675,6465.0,2822.317617,45765.0,18524.166195,50505.0,22528.098381,8430.0,2789.272489,57450.0,25219.495588,...,4650.0,1807.317228,64635.0,27118.997672,4740.0,1824.514606,41190.0,13817.778604,6705.0,2637.722707
175676,7725.0,3085.881665,45735.0,20028.965065,24030.0,10193.354166,23640.0,10681.441361,57855.0,25297.044348,...,4635.0,1981.916417,62775.0,29347.144545,4725.0,1975.152094,39915.0,16853.690903,6645.0,2488.146655
175677,7650.0,3636.386829,45810.0,17385.362109,18000.0,6288.798710,38730.0,16854.617167,59265.0,23744.529706,...,4635.0,2003.650515,62595.0,29119.896786,3105.0,1308.767508,39375.0,18577.764183,6645.0,2787.186256
175678,7635.0,2981.949705,46065.0,18604.384766,8415.0,2868.802547,39315.0,18257.810943,58860.0,21173.453924,...,4620.0,1985.184652,61470.0,25781.111189,2745.0,1154.465979,39315.0,16184.443484,6645.0,3169.747109


In [23]:
send_targets_to_csv.to_csv('Y.csv')